In [ ]:
!pip install stumpy

     |████████████████████████████████| 118 kB 5.2 MB/s 
     |████████████████████████████████| 38.1 MB 239 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
a = []
while(1):
    a.append('1')

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import pathlib
import tqdm
import stumpy

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rootpath = pathlib.Path('/content/drive/MyDrive')
txt_dirpath = rootpath / 'AICSE Project Data'

In [ ]:
import os
os.chdir("/content/drive/My Drive")
!ls

'AICSE Project Data'	 'Getting started.pdf'
'bank statement.pdf'	  kddcup.data_10_percent_corrected
'Colab Notebooks'	  Pickle_Files
'Dublu Bio-data.docx'	  vMalConv
'financial support.pdf'


In [ ]:
# Parameter setting
min_window_size = 40
max_window_size = 800
growth_rate = 1.1
denom_threshold = 0.1
upper_threshold = 0.75
lower_threshold = 0.25
const_threshold = 0.05
min_coef = 0.5
small_quantile = 0.1
padding_length = 3
train_length = 10
use_gpu = False

In [ ]:
# Determine window sizes
size = int(np.log(max_window_size / min_window_size) / np.log(growth_rate)) + 1
rates = np.full(size, growth_rate) ** np.arange(size)
ws = (min_window_size * rates).astype(int)

In [ ]:
# Select stump function
if use_gpu:
    stump = stumpy.gpu_stump
else:
    stump = stumpy.stump

In [ ]:
# Anomaly score names
names = [
    'orig_p2p',
    'diff_p2p',
    'acc_p2p',
    'orig_p2p_inv',
    'diff_small',
    'acc_std',
    'acc_std_inv',
    'orig_mp_novelty',
    'orig_np_novelty',
    'orig_mp_outlier',
    'orig_np_outlier',
]


In [ ]:
def compute_score(X, number, split, w):
        
    # original time series (orig)
    seq = pd.DataFrame(X, columns=['orig'])
    
    # velocity (diff) and acceleration (acc)
    seq['diff'] = seq['orig'].diff(1)
    seq['acc'] = seq['diff'].diff(1)
        
    # standard deviation (std)
    for name in ['orig', 'acc']:
        seq[f'{name}_std'] = seq[name].rolling(w).std().shift(-w)
    
    # peak-to-peak (p2p)
    for name in ['orig', 'diff', 'acc']:
        rolling_max = seq[name].rolling(w).max()
        rolling_min = seq[name].rolling(w).min()
        seq[f'{name}_p2p'] = (rolling_max - rolling_min).shift(-w)
    
    # diff small
    diff_abs = seq['diff'].abs()
    cond = diff_abs <= diff_abs.quantile(small_quantile)
    seq['diff_small'] = cond.rolling(w).mean().shift(-w)
    
    # inverse (inv)
    for name in ['orig_p2p', 'acc_std']:
        numer = seq[name].mean()
        denom = seq[name].clip(lower=numer * denom_threshold)
        seq[f'{name}_inv'] = numer / denom
    
    # coef for penalizing subsequences with little change
    name = 'orig_p2p'
    mean = seq[name].mean()
    upper = mean * upper_threshold
    lower = mean * lower_threshold
    const = mean * const_threshold
    seq['coef'] = (seq[name] - lower) / (upper - lower)
    seq['coef'].clip(upper=1.0, lower=0.0, inplace=True)
    cond = (seq[name] <= const).rolling(2 * w).max().shift(-w) == 1
    seq.loc[cond, 'coef'] = 0.0
        
    # matrix profile value (mpv) and index (mpi)
    mpv = {}
    mpi = {}
    for mode in ['train', 'join', 'all']:
        if mode == 'train':
            mp = stump(X[:split], w)
        elif mode == 'join':
            mp = stump(X[split:], w, X[:split], ignore_trivial=False)
        elif mode == 'all':
            mp = stump(X, w)
        mpv[mode] = mp[:, 0].astype(float)
        mpi[mode] = mp[:, 1].astype(int)
        
    # matrix profile (mp) and normalized profile (np) for novelty detection (AB-join)
    numer = mpv['join']
    denom = mpv['train'][mpi['join']]
    begin = split
    end = begin + len(numer) - 1
    numer *= seq.loc[begin:end, 'coef'].values
    seq.loc[begin:end, 'orig_mp_novelty'] = numer
    with np.errstate(all='ignore'):
        seq.loc[begin:end, 'orig_np_novelty'] = numer / denom
    seq['orig_np_novelty'].clip(upper=1 / denom_threshold, inplace=True)

    # matrix profile (mp) and normalized profile (np) for outlier detection (self-join)
    numer = mpv['all']
    denom = mpv['all'][mpi['all']]
    begin = 0
    end = begin + len(numer) - 1
    numer *= seq.loc[begin:end, 'coef'].values
    seq.loc[begin:end, 'orig_mp_outlier'] = numer
    with np.errstate(all='ignore'):
        seq.loc[begin:end, 'orig_np_outlier'] = numer / denom
    seq['orig_np_outlier'].clip(upper=1 / denom_threshold, inplace=True)
    
    # Smooth and mask anomaly score
    padding = w * padding_length
    seq['mask'] = 0.0
    seq.loc[seq.index[w:-w-padding], 'mask'] = 1.0
    seq['mask'] = seq['mask'].rolling(padding, min_periods=1).sum() / padding
    for name in names:
        seq[f'{name}_score'] = seq[name].rolling(w).mean() * seq['mask']
    
    return seq

In [ ]:
# Evaluate anomaly score for each time series
results = []
for txt_filepath in sorted(txt_dirpath.iterdir()):
    
    # Load time series
    X = np.loadtxt(txt_filepath)
    
    number = txt_filepath.stem.split('_')[0]
    split = int(txt_filepath.stem.split('_')[-1])
    print(f'\n{txt_filepath.name} {split}/{len(X)}', flush=True)
    
    # Evaluate anomaly score for each window size w
    for w in tqdm.tqdm(ws):
        
        # Skip long subsequence
        if w * train_length > split:
            continue
            
        # Compute anomaly score
        seq = compute_score(X, number, split, w)
        
        # Skip if coef is small
        if seq['coef'].mean() < min_coef:
            continue
            
        # Evaluate anomaly score
        for name in names:
            
            # Copy anomaly score
            y = seq[f'{name}_score'].copy()
            
            # Find local maxima
            cond = (y == y.rolling(w, center=True, min_periods=1).max())
            y.loc[~cond] = np.nan
            
            # Find 1st peak
            index1 = y.idxmax()
            value1 = y.max()
            
            # Skip if all score is NaN
            if not np.isfinite(value1):
                continue
                
            # Skip if train data has 1st peak
            begin = index1 - w
            end = index1 + w
            if begin < split:
                continue

            # Find 2nd peak
            y.iloc[begin:end] = np.nan
            index2 = y.idxmax()
            value2 = y.max()
            
            # Skip if 2nd peak height is zero
            if value2 == 0:
                continue
            
            # Evaluate rate of 1st peak height to 2nd peak height
            rate = value1 / value2
            results.append([number, w, name, rate, begin, end, index1, value1, index2, value2])


results = pd.DataFrame(results, columns=['number', 'w', 'name', 'rate', 'begin', 'end', 'index1', 'value1', 'index2', 'value2'])

# Make submission csv
submission = results.loc[results.groupby('number')['rate'].idxmax(), 'index1']
submission.index = np.arange(len(submission)) + 1
submission.name = 'location'
submission.index.name = 'No.'
submission.to_csv('result.csv')


001_UCR_Anomaly_35000.txt 35000/79795


100%|██████████| 32/32 [17:13<00:00, 32.30s/it]



002_UCR_Anomaly_35000.txt 35000/80001


100%|██████████| 32/32 [17:13<00:00, 32.29s/it]



003_UCR_Anomaly_35000.txt 35000/80000


...

100%|██████████| 32/32 [24:48<00:00, 46.51s/it]



074_UCR_Anomaly_4000.txt 4000/36001


100%|██████████| 32/32 [02:09<00:00,  4.03s/it]



075_UCR_Anomaly_4000.txt 4000/30000


100%|██████████| 32/32 [01:31<00:00,  2.85s/it]


In [ ]:
os.chdir("/content/drive/My Drive")
!ls


'AICSE Project Data'   'financial support.pdf'		   result.csv
'bank statement.pdf'   'Getting started.pdf'		   vMalConv
'Colab Notebooks'       kddcup.data_10_percent_corrected
'Dublu Bio-data.docx'   Pickle_Files


In [ ]:
results.head()

,number,w,name,rate,begin,end,index1,value1,index2,value2
0,001,40,orig_p2p,1.126924,66828,66908,66868,1474.047578,9025,1308.027334
1,001,40,acc_p2p,1.006119,61448,61528,61488,1108.742539,25068,1101.999579
2,001,40,diff_small,1.089958,56861,56941,56901,0.325625,56114,0.298750
3,001,40,orig_mp_novelty,1.077604,66357,66437,66397,1.504859,39327,1.396486
4,001,40,orig_np_novelty,1.105841,76724,76804,76764,2.147368,66887,1.941841


In [ ]:
results = pd.DataFrame(results, columns=['number', 'w', 'name', 'rate', 'begin', 'end', 'index1', 'value1', 'index2', 'value2'])

# Make submission csv
submission = results.loc[results.groupby('number')['rate'].idxmax(), 'index1']
submission.index = np.arange(len(submission)) + 1
submission.name = 'location'
submission.index.name = 'No.'
submission.to_csv('result.csv')


NameError: ignored